# 🔎 01 Data Exploration: Chat & Activity Logs
*Case:* The Cyberbullying Case (CIS-3345)
*Analyst:* Luis / Nicholas
*Objective:* Ingest raw evidence files (Discord SQLite DB and Steam Text Logs), normalize timestamps, and filter for key harassment terms.

## 🎯 Goals
1. Connect to the messages.db SQLite database.
2. Convert Unix timestamps to human-readable UTC.
3. Search for specific keywords: "Jane Doe", "meme", "stupid", "ugly".
4. Export a cleaned CSV for the final report.

In [16]:
import sqlite3
import pandas as pd
import os
import glob
from datetime import datetime
# --- CONFIGURATION ---
# Paths are relative to the 'notebooks' folder
DISCORD_DB_PATH = '../evidence/discord/messages.db'
STEAM_LOG_DIR = '../evidence/steam/'
OUTPUT_DIR = '../reports/data_exports/'

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("✅ Environment Setup Complete.")
print(f"📂 Looking for Evidence in: {os.path.abspath('../evidence/')}")

✅ Environment Setup Complete.
📂 Looking for Evidence in: /evidence


## 📘 Section 1: Discord Forensic Analysis
We will connect to the messages.db file extracted from AppData\Roaming\Discord.
Our focus is identifying messages sent by *John Smith* (User ID: YOUR_SUSPECT_ID_HERE) during October 2025.

In [17]:
def get_discord_messages(db_path):
    """
    Connects to the Discord SQLite DB and retrieves messages.
    """
    if not os.path.exists(db_path):
        print(f"⚠️ WARNING: Database not found at {db_path}. Please check file path.")
        # returning an empty dataframe for demonstration purposes so code doesn't break
        return pd.DataFrame(columns=['timestamp', 'author_id', 'content', 'channel_id'])

    conn = sqlite3.connect(db_path)

    # Query to select relevant columns
    query = """
    SELECT
        timestamp,
        author_id,
        channel_id,
        content,
        attachments
    FROM messages
    """

    df = pd.read_sql_query(query, conn)
    conn.close()

    return df

# 1. Load Data
df_discord = get_discord_messages(DISCORD_DB_PATH)

# 2. Clean Data (Convert Timestamp from Unix Milliseconds to Datetime)
# Note: Discord uses Snowflakes or Unix MS. Adjust logic if needed.
if not df_discord.empty:
    df_discord['datetime'] = pd.to_datetime(df_discord['timestamp'], unit='ms')

    # 3. Preview Data
    print(f"✅ Loaded {len(df_discord)} messages from Discord.")
    display(df_discord.head())
else:
    print("❌ No Discord data loaded.")

⚠️ WARNING: Database not found at ../evidence/discord/messages.db. Please check file path.
❌ No Discord data loaded.


## 🔎 Section 2: Keyword Search (The "Smoking Gun")
We will now filter the dataset for the specific harassment keywords identified in the case complaint:
* "Jane Doe"
* "meme"
* "stupid"
* "ugly"

In [18]:
# List of keywords (Case Insensitive)
keywords = ['Jane Doe', 'meme', 'stupid', 'ugly', 'kill', 'hate']

# Create a regex pattern to search for any of these words
pattern = '|'.join(keywords)

if not df_discord.empty:
    # Filter: Content contains keyword OR Attachment contains keyword
    flagged_messages = df_discord[
        df_discord['content'].str.contains(pattern, case=False, na=False) |
        df_discord['attachments'].str.contains(pattern, case=False, na=False)
    ].copy()

    print(f"🚩 Found {len(flagged_messages)} flagged messages.")

    # Display the evidence
    display(flagged_messages[['datetime', 'author_id', 'content']])

    # Export to CSV for the report
    csv_path = os.path.join(OUTPUT_DIR, 'discord_flagged_evidence.csv')
    flagged_messages.to_csv(csv_path, index=False)
    print(f"💾 Evidence saved to: {csv_path}")


## 📗 Section 3: Steam Log Analysis
Steam logs are stored as plain text files. We will iterate through the directory, reading line by line to find mentions of the video or victim.

In [19]:
steam_hits = []

# Walk through the Steam directory
if os.path.exists(STEAM_LOG_DIR):
    for filepath in glob.glob(os.path.join(STEAM_LOG_DIR, "*.txt")):
        with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                # Check for keywords
                for kw in keywords:
                    if kw.lower() in line.lower():
                        steam_hits.append({
                            'source_file': os.path.basename(filepath),
                            'line_number': i + 1,
                            'keyword': kw,
                            'content': line.strip()
                        })
else:
    print(f"⚠️ Steam directory not found at {STEAM_LOG_DIR}")

# Convert to DataFrame for nice display
df_steam = pd.DataFrame(steam_hits)

if not df_steam.empty:
    print(f"🚩 Found {len(df_steam)} hits in Steam logs.")
    display(df_steam)

    # Export
    steam_csv_path = os.path.join(OUTPUT_DIR, 'steam_flagged_evidence.csv')
    df_steam.to_csv(steam_csv_path, index=False)
else:
    print("No keywords found in Steam logs (or no logs present).")

⚠️ Steam directory not found at ../evidence/steam/
No keywords found in Steam logs (or no logs present).
